<a href="https://colab.research.google.com/github/ltpdo/grade-subject/blob/precessing/%E7%A7%91%E7%9B%AE%E3%81%94%E3%81%A8%E3%81%AE%E4%BB%95%E5%88%86%E3%81%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import seaborn as sns

# グラフの日本語化
!pip install japanize_matplotlib
import japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=f62da27ac68a13497a56ca74c2416b8f5c0d5de8131fa7c23b2d2c034c4f2924
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [ ]:
# 機械学習のインポート
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingClassifier, AdaBoostRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# データの前処理

In [ ]:
# Google Driveとの接続
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/')

subject_2021 = pd.read_excel("履修成績データ2021.xlsx")
subject_2021 = subject_2021.drop(columns = ["年度", "学期", "単位数", "成績"])

subject_2021

Mounted at /content/drive


,学生番号,科目名,GP
0,21001,ゲームプログラミング演習,4
1,21001,Reading & WritingⅢ,3
2,21001,Listening & SpeakingⅢ,3
3,21001,韓国語Ⅰ,4
4,21001,計算機構成論Ⅱ,4
...,...,...,...
6870,21140,微分積分Ⅰ,1
6871,21140,離散数学Ⅱ,4
6872,21140,倫理学,2
6873,21140,機械工学概論,3


In [ ]:
# 科目名ごとに学生番号をカウント
subject_count = subject_2021.groupby('科目名')['学生番号'].nunique().reset_index()
count_by_subject = subject_count.rename(columns = {'学生番号': '履修者数'})
count_by_subject = count_by_subject.set_index('科目名')
count_by_subject

,履修者数
科目名,
Advanced Reading & WritingⅠ,2
Advanced Reading & WritingⅡ,3
Domestic Job Training,1
Four SkillsⅠ,13
Four SkillsⅡ,11
...,...
韓国語会話Ⅱ,3
音楽概論,6
ＡＩプログラミング演習,131


In [ ]:
# 科目名ごとに成績をカウント
gp_count_by_subject = subject_2021.groupby(['科目名', 'GP']).size().unstack(fill_value=0)
# 列の値を辞書で置き換える
gp_count_by_subject.columns = ['D', 'C', 'B', 'A', 'S']
gp_count_by_subject

,D,C,B,A,S
科目名,,,,,
Advanced Reading & WritingⅠ,0,0,1,1,0
Advanced Reading & WritingⅡ,1,0,2,0,0
Domestic Job Training,0,0,0,1,0
Four SkillsⅠ,1,0,0,1,11
Four SkillsⅡ,0,0,2,3,6
...,...,...,...,...,...
韓国語会話Ⅱ,0,0,2,0,1
音楽概論,1,0,0,2,3
ＡＩプログラミング演習,5,15,42,45,24


In [ ]:
# 結合
data_2021 = pd.merge(count_by_subject, gp_count_by_subject, left_index=True, right_index=True)
data_2021 = data_2021[data_2021["履修者数"] > 100]
data_2021 = data_2021.drop(index = [
    "Listening & SpeakingⅠ", "Listening & SpeakingⅡ", "Listening & SpeakingⅢ", "Listening & SpeakingⅣ",
    "Reading & WritingⅠ", "Reading & WritingⅡ", "Reading & WritingⅢ", "Reading & WritingⅣ"])
data_2021

,履修者数,D,C,B,A,S
科目名,,,,,,
キャリア形成基礎論,140,13,87,34,6,0
コンピュータネットワーク,104,23,15,17,27,22
データ構造とアルゴリズムⅠ,140,8,18,39,52,23
データ構造とアルゴリズムⅠ演習,132,9,0,5,6,112
ハードウェア実験Ⅰ,108,24,1,13,42,28
ハードウェア設計Ⅰ,127,15,18,24,27,43
ハードウェア設計Ⅱ,121,13,12,16,38,42
プログラミング入門,140,5,6,9,53,71
プログラミング基礎Ⅰ,140,3,19,11,29,79


In [ ]:
# 使用するデータを保存
data_2021.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/データフレーム/科目データ.csv')

# 機械学習

In [ ]:
data_21 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/データフレーム/科目データ(成績評価方法).csv')
data_21

,科目名,履修者数,D,C,B,A,S,テストの割合,小テストの割合,演習(レポート含む)
0,キャリア形成基礎論,140,13,87,34,6,0,40,0,60
1,コンピュータネットワーク,104,23,15,17,27,22,50,0,50
2,データ構造とアルゴリズムⅠ,140,8,18,39,52,23,60,10,30
3,データ構造とアルゴリズムⅠ演習,132,9,0,5,6,112,60,10,30
4,ハードウェア実験Ⅰ,108,24,1,13,42,28,0,0,100
5,ハードウェア設計Ⅰ,127,15,18,24,27,43,75,25,0
6,ハードウェア設計Ⅱ,121,13,12,16,38,42,75,25,0
7,プログラミング入門,140,5,6,9,53,71,0,0,100
8,プログラミング基礎Ⅰ,140,3,19,11,29,79,60,10,30
9,プログラミング基礎Ⅱ,139,7,20,8,26,78,60,10,30
